<a href="https://colab.research.google.com/github/graemet-umich/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Vine Review Analysis
M16-Amazon-Challenge: Deliverable 2

In [1]:
# Install Spark
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
# !wget -q https://archive.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Initialize a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:14 http

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-10-14 22:34:30--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.59MB/s    in 0.2s    

2022-10-14 22:34:30 (5.59 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
#    df = spark.read.format("jdbc").option("url", "jdbc:postgresql://localhost:5432/dezyre_new") \
#    .option("driver", "org.postgresql.Driver").option("dbtable", "drivers_data") \
#    .option("user", "hduser").option("password", "bigdata").load()

# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')

# Configure settings for RDS
mode = "read"
jdbc_url="jdbc:postgresql://dataviz.cob8vbfdaivl.us-east-1.rds.amazonaws.com:5432/M16-Amazon-Challenge"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [14]:
# Read vine_df from table in RDS
vine_df = spark.read.jdbc(url=jdbc_url, table='vine_table', properties=config)
print(vine_df.count())
vine_df.show()

2402431
+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R26MV8D0KG6QI6|          5|            0|          0|   N|                Y|
|R1OF8GP57AQ1A0|          5|            0|          0|   N|                Y|
|R3VDC1QB6MC4ZZ|          5|            0|          0|   N|                N|
|R12FA3DCF8F9ER|          5|            0|          0|   N|                Y|
| RTWHVNV6X4CNJ|          5|            0|          0|   N|                Y|
| RIG9AWFOGRDVO|          2|            1|          1|   N|                Y|
|R1S1XSG4ZCHDGS|          5|            1|          1|   N|                Y|
| RB15NBVY5ELVW|          5|            2|          2|   N|                Y|
| R56358YM1ZJ7I|          5|            0|          0|   N|                N|
|R1ODXB3C9UP3NL|          1|            1|          3|  

In [15]:
# get only reviews with at least 20 total votes
total_votes_gteq20_df = vine_df.filter(vine_df.total_votes >= 20)
print(total_votes_gteq20_df.count())
total_votes_gteq20_df.show()

31518
+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R26XTV29GXRB1N|          1|           25|         25|   N|                Y|
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|
|R18LGGZ2ZZ7OEG|          1|            1|         32|   N|                Y|
| R8E7YMXB7V6DA|          1|           44|         46|   N|                N|
|R3N1V525R0YRJQ|          5|           28|         31|   N|                Y|
|R3PO56NBFXIKV0|          3|           97|        106|   N|                Y|
|R33VIWRD2X5IA9|          1|            0|         20|   N|                Y|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|
|R3W32J5C9K0PFN|          5|           29|         31|   N

In [16]:
# get only reviews where the percentage of helpful votes to total votes is at least 50%
helpful_votes_gteq50_df = total_votes_gteq20_df.filter(total_votes_gteq20_df.helpful_votes / total_votes_gteq20_df.total_votes >= 0.5)
print(helpful_votes_gteq50_df.count())
helpful_votes_gteq50_df.show()

28348
+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R26XTV29GXRB1N|          1|           25|         25|   N|                Y|
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|
| R8E7YMXB7V6DA|          1|           44|         46|   N|                N|
|R3N1V525R0YRJQ|          5|           28|         31|   N|                Y|
|R3PO56NBFXIKV0|          3|           97|        106|   N|                Y|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|
|R3W32J5C9K0PFN|          5|           29|         31|   N|                Y|
|R2K2XZK4SBKYJW|          1|           20|         20|   N|                N|
|R3DOS07BIMP4DJ|          2|           32|         42|   N

In [20]:
# get just paid reviews (Vine program participant)
vine_paid_df = helpful_votes_gteq50_df.filter(helpful_votes_gteq50_df.vine == 'Y')
paid_review_count = vine_paid_df.count()
print(paid_review_count)
vine_paid_df.show()

61
+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3OPNTK61FQ7MP|          5|           32|         37|   Y|                N|
|R2TMZ7GV8SPIRV|          4|           23|         23|   Y|                N|
|R3MKO875WGIEBJ|          5|           34|         37|   Y|                N|
| RMDRU8I773X5U|          4|           22|         25|   Y|                N|
| RX88JLUIFA4Q2|          4|           84|         87|   Y|                Y|
| RV3AHTDRHDH7R|          5|          253|        258|   Y|                N|
|R2BDT05RK7TOJ9|          5|           21|         22|   Y|                N|
| R9SNSQTK4TJOG|          2|           11|         20|   Y|                N|
|R13EGJI8C8YJ4Y|          5|           19|         20|   Y|                N|
|R1A76AK493C8P8|          5|           83|         90|   Y|  

In [19]:
# get just paid reviews (Vine program participant)
vine_unpaid_df = helpful_votes_gteq50_df.filter(helpful_votes_gteq50_df.vine == 'N')
unpaid_review_count = vine_unpaid_df.count()
print(unpaid_review_count)
vine_unpaid_df.show()

28287
+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R26XTV29GXRB1N|          1|           25|         25|   N|                Y|
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|
| R8E7YMXB7V6DA|          1|           44|         46|   N|                N|
|R3N1V525R0YRJQ|          5|           28|         31|   N|                Y|
|R3PO56NBFXIKV0|          3|           97|        106|   N|                Y|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|
|R2K2XZK4SBKYJW|          1|           20|         20|   N|                N|
|R3DOS07BIMP4DJ|          2|           32|         42|   N|                N|
|R3W32J5C9K0PFN|          5|           29|         31|   N

In [27]:
# paid review 5-star summary stats
paid_5star_review_count = vine_paid_df.filter(vine_paid_df.star_rating == 5).count()
paid_5star_review_pct = paid_5star_review_count / paid_review_count
print(f"There were {paid_5star_review_count} 5-star paid reviews out of a total of {paid_review_count} paid reviews.")
print(f"The percentage of paid 5-star reviews was {100 * paid_5star_review_count / paid_review_count:.0f}%")

There were 20 5-star paid reviews out of a total of 61 paid reviews.
The percentage of paid 5-star reviews was 33%


In [28]:
# unpaid review 5-star summary stats
unpaid_5star_review_count = vine_unpaid_df.filter(vine_unpaid_df.star_rating == 5).count()
unpaid_5star_review_pct = unpaid_5star_review_count / unpaid_review_count
print(f"There were {unpaid_5star_review_count} 5-star unpaid reviews out of a total of {unpaid_review_count} unpaid reviews.")
print(f"The percentage of unpaid 5-star reviews was {100 * unpaid_5star_review_count / unpaid_review_count:.1f}%")

There were 15689 5-star unpaid reviews out of a total of 28287 unpaid reviews.
The percentage of unpaid 5-star reviews was 55.5%
